In [87]:
import tensorflow as tf
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense
from keras.initializers import GlorotUniform
tf.enable_eager_execution()


ImportError: cannot import name 'GlorotUniform' from 'keras.initializers' (/home/aniket/anaconda3/lib/python3.7/site-packages/keras/initializers.py)

In [2]:
%%bash
pip install keras==2.2.4

In [3]:
def sqr(var1):
    return tf.math.square(var1)

In [4]:
alpha = tf.Variable(1.0)
beta = tf.Variable(1.0)
phi = tf.Variable(1.0)
theta = tf.Variable(1.0)
gamma = tf.Variable(1.0)
r = tf.Variable(1)
t = tf.Variable(1)
rl = tf.Variable(1)

In [5]:
c1 = tf.Variable(1/3)
c2 = tf.Variable(2/3)

In [6]:
sin_alpha = tf.math.sin(alpha)
sin_two_alpha = tf.math.sin(2*alpha)
sin_beta = tf.math.sin(beta)
sin_two_beta = tf.math.sin(2*beta)
sin_phi = tf.math.sin(phi)
sin_two_phi = tf.math.sin(2*phi)
sin_theta = tf.math.sin(theta)
sin_two_theta = tf.math.sin(2*theta)
sin_gamma = tf.math.sin(gamma)
sin_phi_beta_subt = tf.math.sin(phi - beta)


cos_alpha = tf.math.cos(alpha)
cos_beta = tf.math.cos(beta)
cos_two_beta = tf.math.cos(2*beta)
cos_phi = tf.math.cos(phi)
cos_two_phi = tf.math.cos(2*phi)
cos_theta = tf.math.cos(theta)
cos_gamma = tf.math.cos(gamma)
cos_beta_phi_subt = tf.math.cos(beta - phi)
cos_beta_theta_add = tf.math.cos(beta + theta)

cosec_theta = tf.math.acos(theta)
cot_theta = tf.math.atan(theta)
cot_alpha = tf.math.atan(alpha)


In [7]:
Z1 = sqr(sin_alpha)*sqr(sin_theta)*sqr(cos_beta_phi_subt) + sin_two_theta*sin_alpha*cos_beta_phi_subt - c1

Z2 = sqr(sin_alpha)*sqr(cos_theta)*sqr(cos_beta_phi_subt) - sin_alpha*cos_alpha*sin_two_theta*cos_beta_phi_subt + sqr(cos_alpha)*sqr(sin_theta) - c1

Z3 = sqr(sin_alpha)*sqr(cosec_theta)*sqr(sin_phi_beta_subt) - c1

Z4 = sin_two_theta

Z5 = sqr(sin_alpha)*sin_two_phi + sqr(cos_alpha)*sin_two_theta + sin_two_beta*sqr(sin_alpha)*cos_two_phi + sin_two_alpha*cot_theta*(cos_beta*sin_phi + sin_beta*cos_phi)

Z6 = sqr(sin_alpha)*cot_theta*(cos_beta*sin_beta*cos_two_phi - sqr(cos_beta)*sin_two_phi) + sqr(sin_alpha)*sin_phi_beta_subt

Z7 = sqr(sin_alpha)*(sqr(cos_theta)*sqr(cos_beta_theta_add) + sqr(cos_beta*sin_phi + sin_beta) -sqr(sin_beta)*sqr(sin_phi)  + sqr(cos_alpha)*sqr(sin_theta) - c2  )

Z8 = sqr(cos_beta)*sqr(sin_alpha)

Z9 = sqr(sin_alpha) * ( sin_beta*cos_beta*(sqr(cosec_theta)*cos_two_phi + cosec_theta*cot_theta + cos_two_phi) + sin_beta*cot_alpha*cot_theta*cos_phi - sin_phi*cos_phi*cos_two_beta*cot_theta*(cot_theta - cosec_theta))

In [8]:
neural_net = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(8,)),
    tf.keras.layers.Dense(124, activation = tf.nn.relu),
    tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)
])


In [9]:
def u(t, r, alpha, beta, theta, phi, gamma, rl):
    u = neural_net([t, r, alpha, beta, theta, phi, gamma, rl], weights, biases)
    return u    


In [10]:
def f(u):
    u_t = tf.gradients(u, t)[0]
    u_r = tf.gradients(u, r)[0]
    u_rr = tf.gradients(u_r, r)[0]
    u_sq_rr = tf.gradients(sqr(r)*u_r, r)[0]
    
    u_th = tf.gradients(u, theta)[0]
    u_th = tf.gradients(u_th, theta)[0]
    u_th_sin = tf.gradients(tf.math.sin(theta)*u_th, theta)[0]
    u_th_r = tf.gradients(u_th, r)[0]
    u_th_ph = tf.gradients(u_th, phi)[0]
    
    u_ph = tf.gradients(u, phi)[0]
    u_phph = tf.gradients(u_ph, phi)[0]
    u_ph_r = tf.gradients(u_ph, r)[0]
    
    term1 = (1/sqr(r))*(u_sq_rr)
    term2 = (1/sqr(r*sin_theta))*u_th_sin
    term3 = (1/sqr(r*sin_theta))*u_phph
    del_sqr_spherical = term1 + term2 + term3
    
    Z = Z1*u_rr + (1/sqr(r))*(Z2*u_thth + Z3*u_phph + Z4*u_th_r - Z5*u_ph_r - Z6*u_th_ph + Z8*u_th + Z9*u_phi)+(1/r)*Z7*u_r
    f = u_t - del_sqr_spherical - Z               
    return f               

In [11]:
def loss(u):
    u_t = tf.gradients(u, t)[0]
    u_r = tf.gradients(u, r)[0]
    u_rr = tf.gradients(u_r, r)[0]
    u_sq_rr = tf.gradients(sqr(r)*u_r, r)[0]
    
    u_th = tf.gradients(u, theta)[0]
    u_th = tf.gradients(u_th, theta)[0]
    u_th_sin = tf.gradients(tf.math.sin(theta)*u_th, theta)[0]
    u_th_r = tf.gradients(u_th, r)[0]
    u_th_ph = tf.gradients(u_th, phi)[0]
    
    u_ph = tf.gradients(u, phi)[0]
    u_phph = tf.gradients(u_ph, phi)[0]
    u_ph_r = tf.gradients(u_ph, r)[0]
    
    term1 = (1/sqr(r))*(u_sq_rr)
    term2 = (1/sqr(r*sin_theta))*u_th_sin
    term3 = (1/sqr(r*sin_theta))*u_phph
    del_sqr_spherical = term1 + term2 + term3
    
    Z = Z1*u_rr + (1/sqr(r))*(Z2*u_thth + Z3*u_phph + Z4*u_th_r - Z5*u_ph_r - Z6*u_th_ph + Z8*u_th + Z9*u_phi)+(1/r)*Z7*u_r
    f = u_t - del_sqr_spherical - Z               
    return f               

In [88]:
class model:
    
    def __init__(self):
        xavier=tf.keras.initializers.GlorotUniform()
        self.l1=tf.keras.layers.Dense(64,kernel_initializer=xavier,activation=tf.nn.relu,input_shape=[1])
        self.l2=tf.keras.layers.Dense(64,kernel_initializer=xavier,activation=tf.nn.relu)
        self.out=tf.keras.layers.Dense(1,kernel_initializer=xavier)
        self.train_op = tf.keras.optimizers.Adagrad(learning_rate=0.1)
        
    # Running the model
    def run(self,X):
        boom=self.l1(X)
        boom1=self.l2(boom)
        boom2=self.out(boom1)
        return boom2
      
    #Custom loss fucntion
    def get_loss(self,X,Y):
        boom=self.l1(X)
        boom1=self.l2(boom)
        boom2=self.out(boom1)
        return f(boom2)
      
    # get gradients
    def get_grad(self,X,Y):
        with tf.GradientTape() as tape:
            tape.watch(self.l1.variables)
            tape.watch(self.l2.variables)
            tape.watch(self.out.variables)
            L = self.get_loss(X,Y)
            g = tape.gradient(L, [self.l1.variables[0],self.l1.variables[1],self.l2.variables[0],self.l2.variables[1],self.out.variables[0],self.out.variables[1]])
        return g
      
    # perform gradient descent
    def network_learn(self,X,Y):
        g = self.get_grad(X,Y)
        self.train_op.apply_gradients(zip(g, [self.l1.variables[0],self.l1.variables[1],self.l2.variables[0],self.l2.variables[1],self.out.variables[0],self.out.variables[1]]))



In [160]:
input_shape=(8,1)
model = tf.keras.Sequential([
  tf.keras.layers.Dense(8, activation=tf.nn.relu, input_shape=input_shape),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(1,activation=tf.nn.softmax)
])


In [139]:
inp = tf.Variable(np.random.normal(size=(1,8,1)), dtype=tf.float32)

with tf.GradientTape() as tape:
    preds = model(inp)

grads = tape.gradient(preds, inp)

In [164]:
inputs = np.array([1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])
#inputs = tf.reshape(inputs,[1,8,1])
#inputs = tf.Variable(inputs)
#inputs = tf.cast(inputs,tf.float32)
t = tf.Variable(inputs[0])
r = tf.Variable(inputs[1])
alpha= tf.Variable(inputs[2])
beta= tf.Variable(inputs[3])
theta= tf.Variable(inputs[4])
phi= tf.Variable(inputs[5])
gamma= tf.Variable(inputs[6])
rl= tf.Variable(inputs[7])



with tf.GradientTape() as tape1:
    inp = tf.reshape(tf.stack([t, r, alpha, beta, theta, phi, gamma, rl]),[1,8,1])
    inp = tf.cast(inp,tf.float32)
    tape1.watch([t, r, alpha, beta, theta, phi, gamma, rl])
    preds = model(inp)

#print(tape1.gradient(tf.convert_to_tensor(preds), inputs))

d_t,d_r,d_al,d_bt,d_th,d_ph,d_gm,d_rl = tape1.gradient(preds, [t, r, alpha, beta, theta, phi, gamma, rl])

print (d_t)


tf.Tensor(0.0, shape=(), dtype=float64)


In [143]:
print(d_t_t)

None


In [45]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [46]:
num_samples = 1000
x_size = [1000,8,1]

In [47]:
x_size

[1000, 8, 1]

In [48]:
points = tf.random.uniform(shape=x_size, dtype=tf.dtypes.float32, seed=10) 

In [49]:
labels = tf.random.uniform(shape=[num_samples], dtype=tf.dtypes.float32, seed=10) 

In [50]:
train_dataset = tuple(zip(points, labels))

In [107]:
def grad(model, inputs, targets):
    inputs = tf.reshape(inputs,[1,8,1])
    with tf.GradientTape() as tape2:
        with tf.GradientTape() as tape1:
            tape1.watch(inputs)
            preds = model(inputs)
        print(tape1.gradient(tf.convert_to_tensor(preds), inputs))
        d_t,d_r,d_al,d_bt,d_th,d_ph,d_gm,d_rl = tape1.gradient(tf.convert_to_tensor(preds), inputs)
    d_r_r = tape2.gradient(d_r, r)
    d_sq_r_r = tape2.gradient(sqr(r)*d_r, r)
    d_th_sin = tape2.gradient(tf.math.sin(theta)*d_th, theta)
    d_th_r = tape2.gradient(d_th, r)
    d_th_ph = tape2.gradient(d_th, phi)
    d_th_th = tape2.gradient(d_th, theta)
    d_ph_ph = tape2.gradient(d_ph, phi)
    d_ph_r = tape2.gradient(d_ph, r)
    term1 = (1/sqr(r))*(d_sq_r_r)
    term2 = (1/sqr(r*sin_theta))*d_th_sin
    term3 = (1/sqr(r*sin_theta))*d_ph_ph
    del_sqr_spherical = term1 + term2 + term3

    Z = Z1*u_rr + (1/sqr(r))*(Z2*d_th_th + Z3*d_ph_ph + Z4*d_th_r - Z5*d_ph_r - Z6*d_th_ph + Z8*d_th + Z9*d_phi)+(1/r)*Z7*d_r
    loss_value = d_t - del_sqr_spherical - Z  
    return loss_value,tape.gradient(loss_value, model.trainable_variables)
            
            
    



In [108]:
## Note: Rerunning this cell uses the same model variables

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
      epoch_loss_avg = tf.keras.metrics.Mean()
      epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


  # Training loop - using batches of 32
      for x, y in train_dataset:
        # Optimize the model1) 
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        # Track progress
        epoch_loss_avg(loss_value)
        
        # Add current batch loss
        # Compare predicted label to actual label
        # training=True is needed only if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        epoch_accuracy(y, model(x, training=True))
        
        # End epoch
        train_loss_results.append(epoch_loss_avg.result())
        train_accuracy_results.append(epoch_accuracy.result())

        if epoch % 50 == 0:
            print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                    epoch_loss_avg.result()))


tf.Tensor(
[[[0.5331018]
  [0.5331018]
  [0.5331018]
  [0.5331018]
  [0.5331018]
  [0.5331018]
  [0.5331018]
  [0.5331018]]], shape=(1, 8, 1), dtype=float32)


RuntimeError: GradientTape.gradient can only be called once on non-persistent tapes.

In [ ]:
model

model.compile(optimizer = 'adam', loss = custom_loss, metrics = ['accuracy'])

In [ ]:
x=[1,2,3,4,5,6,7,8,9,10]
x=np.asarray(x,dtype=np.float32).reshape((10,1))
y=[1,4,9,16,25,36,49,64,81,100]
y=np.asarray(y,dtype=np.float32).reshape((10,1))
model=model()
for i in range(100):
    model.network_learn(x,y)